## 변수는 다음과 같이 변환하다.
- cloud dummy추가
- 화씨전환
- month dummy 추가 -> good!!
- WindDirection은 제거해준다(plot근거로)
- temp^2변수를 추가해준다(plot 기준 abs보다 타당하다고 고려)
- temp^3변수를 추가해보자


# data의 AR을 제거해주기 위해 shuffle하였다
## 각 시간별 지역별 모델은 crossvalidation기준으로 선정
### us
- 18,19 -> RF        /  나머지 ->  LR

### dj_or
- 17 -> RF           /       나머지 ->  LR


### dj_wh
- 17,18 -> RF           /              나머지 ->  LR

### dj_ft
- 17,18 -> RF            /        나머지 ->LR


## train 및 test 생성

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go

from datetime import timedelta

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score

In [2]:
fcst_us=pd.read_csv("ulsan_fcst_best_new.csv")
fcst_dj=pd.read_csv("DJ_fcst_best_new.csv")
energy=pd.read_csv('energy.csv')

In [3]:
# 시간으로 분류를해본다

energy['date'] = energy['time'].apply(lambda x: x.split()[0])
energy['time'] = energy['time'].apply(lambda x: x.split()[1])
energy['time'] = energy['time'].str.rjust(8,'0') # 한자릿수 시간 앞에 0 추가 ex) 3시 -> 03시

# 24시를 00시로 바꿔주기
energy.loc[energy['time']=='24:00:00','time'] = '00:00:00'
energy['time'] = energy['date'] + ' ' + energy['time']
energy['time'] = pd.to_datetime(energy['time'])
energy.loc[energy['time'].dt.hour==0,'time'] += timedelta(days=1)



energy['year']=energy['time'].dt.year
energy['month']=energy['time'].dt.month
energy['day']=energy['time'].dt.day
energy['hour']=energy['time'].dt.hour

energy['day_int']=energy['year']*10000+energy['month']*100+energy['day']
energy['day_hour_int']=energy['day_int']*100+energy['hour']

energy


,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan,date,year,month,day,hour,day_int,day_hour_int
0,2018-03-01 01:00:00,0.0,0.0,0,0,2018-03-01,2018,3,1,1,20180301,2018030101
1,2018-03-01 02:00:00,0.0,0.0,0,0,2018-03-01,2018,3,1,2,20180301,2018030102
2,2018-03-01 03:00:00,0.0,0.0,0,0,2018-03-01,2018,3,1,3,20180301,2018030103
3,2018-03-01 04:00:00,0.0,0.0,0,0,2018-03-01,2018,3,1,4,20180301,2018030104
4,2018-03-01 05:00:00,0.0,0.0,0,0,2018-03-01,2018,3,1,5,20180301,2018030105
...,...,...,...,...,...,...,...,...,...,...,...,...
25627,2021-01-31 20:00:00,0.0,0.0,0,0,2021-01-31,2021,1,31,20,20210131,2021013120
25628,2021-01-31 21:00:00,0.0,0.0,0,0,2021-01-31,2021,1,31,21,20210131,2021013121
25629,2021-01-31 22:00:00,0.0,0.0,0,0,2021-01-31,2021,1,31,22,20210131,2021013122
25630,2021-01-31 23:00:00,0.0,0.0,0,0,2021-01-31,2021,1,31,23,20210131,2021013123


In [4]:
#소수점 맞춤
fcst_dj['Temperature']=round(fcst_dj['Temperature'])
fcst_us['Temperature']=round(fcst_us['Temperature'])

fcst_dj['Humidity']=round(fcst_dj['Humidity'])
fcst_us['Humidity']=round(fcst_us['Humidity'])


fcst_dj['WindSpeed']=round(fcst_dj['WindSpeed'],1)
fcst_us['WindSpeed']=round(fcst_us['WindSpeed'],1)

In [5]:
fcst_dj

,Forecast_time,Temperature,Humidity,WindSpeed,Cloud
0,2018-03-01 15:00:00,0.0,60.0,7.3,2.0
1,2018-03-01 16:00:00,-1.0,60.0,7.2,2.0
2,2018-03-01 17:00:00,-2.0,60.0,7.2,1.0
3,2018-03-01 18:00:00,-2.0,60.0,7.1,1.0
4,2018-03-01 19:00:00,-2.0,61.0,7.2,1.0
...,...,...,...,...,...
26285,2021-02-28 20:00:00,7.0,67.0,1.1,4.0
26286,2021-02-28 21:00:00,7.0,70.0,1.3,4.0
26287,2021-02-28 22:00:00,7.0,73.0,1.3,4.0
26288,2021-02-28 23:00:00,6.0,76.0,1.3,4.0


In [6]:
# 시간(hour) 변수 생성
fcst_dj['hour']= fcst_dj['Forecast_time'].str.split(' ').str[1].str.split(':').str[0].astype(int)
fcst_dj['date']= fcst_dj['Forecast_time'].str.split(' ').str[0]

# year montj day추가
year_fcst = fcst_dj['date'].str.split('-').str[0].astype(int)
month_fcst = fcst_dj['date'].str.split('-').str[1].astype(int)
day_fcst = fcst_dj['date'].str.split('-').str[2].astype(int)
fcst_dj['day_int']=year_fcst*10000+month_fcst*100+day_fcst

fcst_dj['year']=year_fcst
fcst_dj['month']=month_fcst
fcst_dj['day']=day_fcst

# 정렬을 쉽게하기위한 day_hour_int를 생성
fcst_dj['day_int']=year_fcst*10000+month_fcst*100+day_fcst
fcst_dj['day_hour_int']=fcst_dj['day_int']*100+fcst_dj['hour']

In [7]:
# 시간(hour) 변수 생성
fcst_us['hour']= fcst_us['Forecast_time'].str.split(' ').str[1].str.split(':').str[0].astype(int)
fcst_us['date']= fcst_us['Forecast_time'].str.split(' ').str[0]

# year montj day추가
year_fcst = fcst_us['date'].str.split('-').str[0].astype(int)
month_fcst = fcst_us['date'].str.split('-').str[1].astype(int)
day_fcst = fcst_us['date'].str.split('-').str[2].astype(int)

fcst_us['year']=year_fcst
fcst_us['month']=month_fcst
fcst_us['day']=day_fcst


# 정렬을 쉽게하기위한 day_hour_int를 생성
fcst_us['day_int']=year_fcst*10000+month_fcst*100+day_fcst
fcst_us['day_hour_int']=fcst_us['day_int']*100+fcst_us['hour']

# 변수변환

In [8]:
## 화씨 생성
fcst_dj['Temp_F']=fcst_dj['Temperature']*9/5+32
fcst_us['Temp_F']=fcst_us['Temperature']*9/5+32

## 제곱 온도 생성  &  3제곱 온도 생성
fcst_dj['Temp2']=fcst_dj['Temperature']**2
fcst_us['Temp2']=fcst_us['Temperature']**2

fcst_dj['Temp3']=fcst_dj['Temperature']**3
fcst_us['Temp3']=fcst_us['Temperature']**3


#구름 더미화
fcst_incoded_dj_1=pd.get_dummies(fcst_dj,columns=['Cloud'])
fcst_incoded_us_1=pd.get_dummies(fcst_us,columns=['Cloud'])

# 월별 더미화
fcst_incoded_dj=pd.get_dummies(fcst_incoded_dj_1,columns=['month'])
fcst_incoded_us=pd.get_dummies(fcst_incoded_us_1,columns=['month'])


In [9]:
fcst_incoded_dj.columns

Index(['Forecast_time', 'Temperature', 'Humidity', 'WindSpeed', 'hour', 'date',
       'day_int', 'year', 'day', 'day_hour_int', 'Temp_F', 'Temp2', 'Temp3',
       'Cloud_1.0', 'Cloud_2.0', 'Cloud_3.0', 'Cloud_4.0', 'month_1',
       'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7',
       'month_8', 'month_9', 'month_10', 'month_11', 'month_12'],
      dtype='object')

### 결합해서 데이터 생성

In [10]:
# train_X


train_us=fcst_incoded_us[(fcst_incoded_us['day_int']<=20210131)&(fcst_incoded_us['day_int']>=20180302)]
train_us=train_us.reset_index().drop(['index'],axis=1)

train_dj=fcst_incoded_dj[(fcst_incoded_dj['day_int']<=20210131)&(fcst_incoded_dj['day_int']>=20180302)]
train_dj=train_dj.reset_index().drop(['index'],axis=1)


train_dj_or=train_dj.copy()
train_dj_wh=train_dj.copy()
train_dj_ft=train_dj.copy()


#y값 추가 및 na 값 제거
train_y=energy[(energy['day_int']<=20210131)&(energy['day_int']>=20180302)]
train_y=train_y.reset_index(drop=True)


train_us['energy']=train_y.ulsan
train_us=train_us.dropna(axis=0)

train_dj_or['energy']=train_y.dangjin
train_dj_or=train_dj_or.dropna(axis=0)

train_dj_wh['energy']=train_y.dangjin_warehouse
train_dj_wh=train_dj_wh.dropna(axis=0)

train_dj_ft['energy']=train_y.dangjin_floating
train_dj_ft=train_dj_ft.dropna(axis=0)







In [11]:
train_dj_or

,Forecast_time,Temperature,Humidity,WindSpeed,hour,date,day_int,year,day,day_hour_int,...,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,energy
0,2018-03-02 00:00:00,-2.0,50.0,5.8,0,2018-03-02,20180302,2018,2,2018030200,...,0,0,0,0,0,0,0,0,0,0
1,2018-03-02 01:00:00,-2.0,50.0,5.5,1,2018-03-02,20180302,2018,2,2018030201,...,0,0,0,0,0,0,0,0,0,0
2,2018-03-02 02:00:00,-3.0,53.0,5.3,2,2018-03-02,20180302,2018,2,2018030202,...,0,0,0,0,0,0,0,0,0,0
3,2018-03-02 03:00:00,-3.0,55.0,5.0,3,2018-03-02,20180302,2018,2,2018030203,...,0,0,0,0,0,0,0,0,0,0
4,2018-03-02 04:00:00,-3.0,57.0,4.7,4,2018-03-02,20180302,2018,2,2018030204,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25603,2021-01-31 19:00:00,6.0,78.0,3.5,19,2021-01-31,20210131,2021,31,2021013119,...,0,0,0,0,0,0,0,0,0,0
25604,2021-01-31 20:00:00,6.0,80.0,3.8,20,2021-01-31,20210131,2021,31,2021013120,...,0,0,0,0,0,0,0,0,0,0
25605,2021-01-31 21:00:00,6.0,80.0,4.0,21,2021-01-31,20210131,2021,31,2021013121,...,0,0,0,0,0,0,0,0,0,0
25606,2021-01-31 22:00:00,6.0,80.0,4.7,22,2021-01-31,20210131,2021,31,2021013122,...,0,0,0,0,0,0,0,0,0,0


## SHUFFLE

In [12]:
train_us=train_us.sample(frac=1).reset_index(drop=True)
train_dj_or=train_dj_or.sample(frac=1).reset_index(drop=True)
train_dj_wh=train_dj_wh.sample(frac=1).reset_index(drop=True)
train_dj_ft=train_dj_ft.sample(frac=1).reset_index(drop=True)

In [13]:
# 각 train data 내보내기
#train_us.to_csv("train_month_us.csv",index=False)
#train_dj_or.to_csv("train_month_dj_or.csv",index=False)
#train_dj_wh.to_csv("train_month_dj_wh.csv",index=False)
#train_dj_ft.to_csv("train_month_dj_ft.csv",index=False)

## test 생성
dummy변수의 마지막 변수를 제외-> 최종 변수: 습도,풍속,기온,기온^2,구름-1&2&3, month-1~11

In [14]:
test_us=fcst_incoded_us[(fcst_incoded_us['day_hour_int']>=2021020100)&(fcst_incoded_us['day_hour_int']<=2021022823)]
test_us=test_us.reset_index().drop(['index'],axis=1)
test_us['energy']=0

test_X_us=test_us[['Humidity','WindSpeed','Temperature','Temp2','Temp3','Cloud_1.0','Cloud_2.0','Cloud_3.0',
                  'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11','day_hour_int','hour','energy']]



test_dj=fcst_incoded_dj[(fcst_incoded_dj['day_hour_int']>=2021020100)&(fcst_incoded_dj['day_hour_int']<=2021022823)]
test_dj=test_dj.reset_index().drop(['index'],axis=1)
test_dj['energy']=0

test_X_dj=test_dj[['Humidity','WindSpeed','Temperature','Temp2','Temp3','Cloud_1.0','Cloud_2.0','Cloud_3.0',
                  'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11','day_hour_int','hour','energy']]


In [15]:
test_X_dj.columns==test_X_us.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

## 변수선택 및 최적모형 예측

In [16]:

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score

from sklearn.neighbors import KNeighborsRegressor # k-최근접 이웃 회귀 #분류대신 값을 예측
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge




#개별 모델 성능 (MAE) 확인

knn_model = KNeighborsRegressor()
svr_model = SVR()
lr_model = LinearRegression()
rf_model = RandomForestRegressor()
lgb_model = LGBMRegressor()

In [17]:
PRED_us=pd.DataFrame(np.zeros((28,24)))

### 18,19 -> RF     /  나머지 ->  LR


for h in range(24):
########################  평가기준 아닌 시간은 0########################3    
    if h<=7:
        PRED_us.iloc[:,h]=0
    elif h>=20:
        PRED_us.iloc[:,h]=0

#######################  18,19시 ->  RF ############################        

    elif h>=18:
        data=train_us[train_us['hour']==h].reset_index(drop=True)
        data=data[data['energy']>0]
        
        test_h=test_X_us[test_X_us['hour']==h].reset_index(drop=True)


        train_X=data[['Humidity','WindSpeed','Temperature','Temp2','Temp3','Cloud_1.0','Cloud_2.0','Cloud_3.0',
                  'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11']]

        train_y=data.energy.reset_index(drop=True)
        test=test_h.iloc[:,:-3]



        rf_model = RandomForestRegressor()
        rf_model.fit(train_X,train_y)

        #예측값
        energy=rf_model.predict(test)
        PRED_us.iloc[:,h]=energy
        
        
####################### 나머지-> LR ###############################
    else:
        data=train_us[train_us['hour']==h].reset_index(drop=True)
        data=data[data['energy']>0]
        
        test_h=test_X_us[test_X_us['hour']==h].reset_index(drop=True)


        train_X=data[['Humidity','WindSpeed','Temperature','Temp2','Temp3','Cloud_1.0','Cloud_2.0','Cloud_3.0',
                  'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11']]

        train_y=data.energy.reset_index(drop=True)
        test=test_h.iloc[:,:-3]



        lr_model = LinearRegression()
        lr_model.fit(train_X,train_y)

        #예측값
        energy=lr_model.predict(test)
        PRED_us.iloc[:,h]=energy

In [18]:
PRED_dj_or=pd.DataFrame(np.zeros((28,24)))
# 17-> RF       /       나머지 ->  LR


for h in range(24):
    if h<=7:
        PRED_dj_or.iloc[:,h]=0
    elif h>=20:
        PRED_dj_or.iloc[:,h]=0

#######################  17시 ->  RF ############################        
    elif h==17:
        data=train_dj_or[train_dj_or['hour']==h].reset_index(drop=True)
        data=data[data['energy']>0]
        
        test_h=test_X_dj[test_X_dj['hour']==h].reset_index(drop=True)


        train_X=data[['Humidity','WindSpeed','Temperature','Temp2','Temp3','Cloud_1.0','Cloud_2.0','Cloud_3.0',
                  'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11']]

        train_y=data.energy.reset_index(drop=True)
        test=test_h.iloc[:,:-3]




        rf_model = RandomForestRegressor()
        rf_model.fit(train_X,train_y)

        #예측값
        energy=rf_model.predict(test)
        PRED_dj_or.iloc[:,h]=energy

         
####################### 나머지-> LR ###############################
    
    else:
        data=train_dj_or[train_dj_or['hour']==h].reset_index(drop=True)
        data=data[data['energy']>0]
        
        test_h=test_X_dj[test_X_dj['hour']==h].reset_index(drop=True)


        train_X=data[['Humidity','WindSpeed','Temperature','Temp2','Temp3','Cloud_1.0','Cloud_2.0','Cloud_3.0',
                  'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11']]

        train_y=data.energy.reset_index(drop=True)
        test=test_h.iloc[:,:-3]
        

        lr_model = LinearRegression()
        lr_model.fit(train_X,train_y)

        #예측값
        energy=lr_model.predict(test)
        PRED_dj_or.iloc[:,h]=energy

In [19]:
PRED_dj_wh=pd.DataFrame(np.zeros((28,24)))
# 17,18 -> RF           /              나머지 ->  LR


for h in range(24):
    if h<=7:
        PRED_dj_wh.iloc[:,h]=0
    elif h>=20:
        PRED_dj_wh.iloc[:,h]=0
#######################  17,18시 ->  RF ############################        
    elif h==17:
        data=train_dj_wh[train_dj_wh['hour']==h].reset_index(drop=True)
        data=data[data['energy']>0]
        
        test_h=test_X_dj[test_X_dj['hour']==h].reset_index(drop=True)


        train_X=data[['Humidity','WindSpeed','Temperature','Temp2','Temp3','Cloud_1.0','Cloud_2.0','Cloud_3.0',
                  'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11']]

        train_y=data.energy.reset_index(drop=True)
        test=test_h.iloc[:,:-3]
        


        rf_model = RandomForestRegressor()
        rf_model.fit(train_X,train_y)

        #예측값
        energy=rf_model.predict(test)
        PRED_dj_wh.iloc[:,h]=energy
        
    elif h==18:
        data=train_dj_wh[train_dj_wh['hour']==h].reset_index(drop=True)
        data=data[data['energy']>0]
        
        test_h=test_X_dj[test_X_dj['hour']==h].reset_index(drop=True)


        train_X=data[['Humidity','WindSpeed','Temperature','Temp2','Temp3','Cloud_1.0','Cloud_2.0','Cloud_3.0',
                  'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11']]

        train_y=data.energy.reset_index(drop=True)
        test=test_h.iloc[:,:-3]



        rf_model = RandomForestRegressor()
        rf_model.fit(train_X,train_y)

        #예측값
        energy=rf_model.predict(test)
        PRED_dj_wh.iloc[:,h]=energy

         
####################### 나머지-> LR ###############################
    else:
        data=train_dj_wh[train_dj_wh['hour']==h].reset_index(drop=True)
        data=data[data['energy']>0]
        
        test_h=test_X_dj[test_X_dj['hour']==h].reset_index(drop=True)


        train_X=data[['Humidity','WindSpeed','Temperature','Temp2','Temp3','Cloud_1.0','Cloud_2.0','Cloud_3.0',
                  'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11']]

        train_y=data.energy.reset_index(drop=True)
        test=test_h.iloc[:,:-3]


        lr_model = LinearRegression()
        lr_model.fit(train_X,train_y)

        #예측값
        energy=lr_model.predict(test)
        PRED_dj_wh.iloc[:,h]=energy

In [20]:
PRED_dj_ft=pd.DataFrame(np.zeros((28,24)))
# 17,18 -> RF            /        나머지 ->LR


for h in range(24):
    if h<=7:
        PRED_dj_ft.iloc[:,h]=0
    elif h>=20:
        PRED_dj_ft.iloc[:,h]=0

#######################  17시,18시 ->  RF ############################        

    elif h==17:
        data=train_dj_ft[train_dj_ft['hour']==h].reset_index(drop=True)
        data=data[data['energy']>0]
        
        test_h=test_X_dj[test_X_dj['hour']==h].reset_index(drop=True)


        train_X=data[['Humidity','WindSpeed','Temperature','Temp2','Temp3','Cloud_1.0','Cloud_2.0','Cloud_3.0',
                  'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11']]

        train_y=data.energy.reset_index(drop=True)
        test=test_h.iloc[:,:-3]



        rf_model = RandomForestRegressor()
        rf_model.fit(train_X,train_y)

        #예측값
        energy=rf_model.predict(test)
        PRED_dj_ft.iloc[:,h]=energy


    elif h==18:
        data=train_dj_ft[train_dj_ft['hour']==h].reset_index(drop=True)
        data=data[data['energy']>0]
        
        test_h=test_X_dj[test_X_dj['hour']==h].reset_index(drop=True)


        train_X=data[['Humidity','WindSpeed','Temperature','Temp2','Temp3','Cloud_1.0','Cloud_2.0','Cloud_3.0',
                  'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11']]

        train_y=data.energy.reset_index(drop=True)
        test=test_h.iloc[:,:-3]



        rf_model = RandomForestRegressor()
        rf_model.fit(train_X,train_y)

        #예측값
        energy=rf_model.predict(test)
        PRED_dj_ft.iloc[:,h]=energy
####################### 나머지-> LR ###############################
    else:
        data=train_dj_ft[train_dj_ft['hour']==h].reset_index(drop=True)
        data=data[data['energy']>0]
        
        test_h=test_X_dj[test_X_dj['hour']==h].reset_index(drop=True)


        train_X=data[['Humidity','WindSpeed','Temperature','Temp2','Temp3','Cloud_1.0','Cloud_2.0','Cloud_3.0',
                  'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11']]

        train_y=data.energy.reset_index(drop=True)
        test=test_h.iloc[:,:-3]



        lr_model = LinearRegression()
        lr_model.fit(train_X,train_y)

        #예측값
        energy=lr_model.predict(test)
        PRED_dj_ft.iloc[:,h]=energy

In [21]:
PRED_np_us=PRED_us.to_numpy()
PRED_np_dj_or=PRED_dj_or.to_numpy()
PRED_np_dj_ft=PRED_dj_ft.to_numpy()
PRED_np_dj_wh=PRED_dj_wh.to_numpy()


In [22]:
test_X_dj['dangjin']=PRED_np_dj_or.reshape(-1,1)
test_X_dj['dangjin_warehouse']=PRED_np_dj_wh.reshape(-1,1)
test_X_dj['dangjin_floating']=PRED_np_dj_ft.reshape(-1,1)
test_X_us['ulsan']=PRED_np_us.reshape(-1,1)

In [23]:
test_X_us

,Humidity,WindSpeed,Temperature,Temp2,Temp3,Cloud_1.0,Cloud_2.0,Cloud_3.0,month_1,month_2,...,month_6,month_7,month_8,month_9,month_10,month_11,day_hour_int,hour,energy,ulsan
0,65.0,4.7,11.0,121.0,1331.0,0,0,0,0,1,...,0,0,0,0,0,0,2021020100,0,0,0.00
1,75.0,6.2,11.0,121.0,1331.0,0,0,0,0,1,...,0,0,0,0,0,0,2021020101,1,0,0.00
2,75.0,6.2,11.0,121.0,1331.0,0,0,0,0,1,...,0,0,0,0,0,0,2021020102,2,0,0.00
3,75.0,6.2,11.0,121.0,1331.0,0,0,0,0,1,...,0,0,0,0,0,0,2021020103,3,0,0.00
4,79.0,6.7,12.0,144.0,1728.0,0,0,0,0,1,...,0,0,0,0,0,0,2021020104,4,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,72.0,3.5,14.0,196.0,2744.0,0,0,0,0,1,...,0,0,0,0,0,0,2021022819,19,0,4.11
668,74.0,3.9,14.0,196.0,2744.0,0,0,0,0,1,...,0,0,0,0,0,0,2021022820,20,0,0.00
669,75.0,4.3,14.0,196.0,2744.0,0,0,0,0,1,...,0,0,0,0,0,0,2021022821,21,0,0.00
670,77.0,4.0,14.0,196.0,2744.0,0,0,0,0,1,...,0,0,0,0,0,0,2021022822,22,0,0.00


In [24]:
sub=pd.read_csv('sample_submission.csv')
sub_2=sub.iloc[:671,:]
sub_2
sub_67=sub.iloc[671:,:]
sub.head(10)

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2021-02-01 01:00:00,0,0,0,0
1,2021-02-01 02:00:00,0,0,0,0
2,2021-02-01 03:00:00,0,0,0,0
3,2021-02-01 04:00:00,0,0,0,0
4,2021-02-01 05:00:00,0,0,0,0
5,2021-02-01 06:00:00,0,0,0,0
6,2021-02-01 07:00:00,0,0,0,0
7,2021-02-01 08:00:00,0,0,0,0
8,2021-02-01 09:00:00,0,0,0,0
9,2021-02-01 10:00:00,0,0,0,0


In [25]:
sub_2['dangjin']=list(test_X_dj.iloc[1:,:].dangjin)
sub_2['dangjin_warehouse']=list(test_X_dj.iloc[1:,:].dangjin_warehouse)
sub_2['dangjin_floating']=list(test_X_dj.iloc[1:,:].dangjin_floating)
sub_2['ulsan']=list(test_X_us.iloc[1:,:].ulsan)



sub_final=pd.concat([sub_2,sub_67],axis=0)
sub_final.to_csv('sub_shuffled_best_new.csv',index=False)